In [1]:
import numpy
import pandas
import torch
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pandas.read_json("../datasets/joke_info.json")
df

,joke_id,USF1000_rating,USF1001_rating,USF1002_rating,datasource_joke_id,jokeText,jester,reddit,stupidstuff
0,RE5tz52q,1,8,0,5tz52q,I hate how you cant even say black paint anymo...,0,1,0
1,RE5tz4dd,1,8,0,5tz4dd,What's the difference between a Jew in Nazi Ge...,0,1,0
2,RE5tz319,0,9,8,5tz319,I recently went to America....\n...and being t...,0,1,0
3,RE5tz2wj,3,9,0,5tz2wj,"Brian raises his hand and says, “He’s in Heave...",0,1,0
4,RE5tz1pc,5,3,8,5tz1pc,You hear about the University book store worke...,0,1,0
...,...,...,...,...,...,...,...,...,...
186,SS200,8,10,4,200,A concerned husband went to a doctor to talk a...,0,0,1
187,SS5,0,0,0,5,A girl came skipping home FROM school one day....,0,0,1
188,SS6,0,0,0,6,An airline captain was helping a new blonde fl...,0,0,1
189,SS7,0,0,0,7,A blonde and a brunette decided to rob a bank....,0,0,1


In [3]:
USF_users = pandas.read_csv('../datasets/users.csv')
USF_users

,user_id,name,age,gender,ethnicity,country,location
0,USF1000,Tanya,24,Female,South East Asian,India,San Francisco
1,USF1001,rishab,26,Male,South East Asian,India,San Francisco
2,USF1002,Parisa,26,Female,Middle Eastern,Iran,San Francisco


In [5]:
 df[df.columns[df.columns.str.startswith('USF')]]

,USF1000_rating,USF1001_rating,USF1002_rating
0,1,8,0
1,1,8,0
2,0,9,8
3,3,9,0
4,5,3,8
...,...,...,...
186,8,10,4
187,0,0,0
188,0,0,0
189,0,0,0


Before getting started with this -

we need a metric to score or rate movie
Calculate the score for every movie
Sort the scores and recommend the best rated movie to the users.
We can use the average ratings of the movie as the score but using this won't be fair enough since a movie with 8.9 average rating and only 3 votes cannot be considered better than the movie with 7.8 as as average rating but 40 votes. So, I'll be using IMDB's weighted rating (wr) which is given as :-

 where,

v is the number of votes for the movie;
m is the minimum votes required to be listed in the chart;
R is the average rating of the movie; And
C is the mean vote across the whole report

In [9]:
# in df get average raating for each joke by gettng user ratings by column startwith 'USF'
# then get mean of each row
df['avg_rating'] = df[df.columns[df.columns.str.startswith('USF')]].mean(axis=1)
df['rating_count'] = df[df.columns[df.columns.str.startswith('USF')]].count(axis=1)
df

,joke_id,USF1000_rating,USF1001_rating,USF1002_rating,datasource_joke_id,jokeText,jester,reddit,stupidstuff,avg_rating,rating_count
0,RE5tz52q,1,8,0,5tz52q,I hate how you cant even say black paint anymo...,0,1,0,3.000000,3
1,RE5tz4dd,1,8,0,5tz4dd,What's the difference between a Jew in Nazi Ge...,0,1,0,3.000000,3
2,RE5tz319,0,9,8,5tz319,I recently went to America....\n...and being t...,0,1,0,5.666667,3
3,RE5tz2wj,3,9,0,5tz2wj,"Brian raises his hand and says, “He’s in Heave...",0,1,0,4.000000,3
4,RE5tz1pc,5,3,8,5tz1pc,You hear about the University book store worke...,0,1,0,5.333333,3
...,...,...,...,...,...,...,...,...,...,...,...
186,SS200,8,10,4,200,A concerned husband went to a doctor to talk a...,0,0,1,7.333333,3
187,SS5,0,0,0,5,A girl came skipping home FROM school one day....,0,0,1,0.000000,3
188,SS6,0,0,0,6,An airline captain was helping a new blonde fl...,0,0,1,0.000000,3
189,SS7,0,0,0,7,A blonde and a brunette decided to rob a bank....,0,0,1,0.000000,3


In [7]:
C= df['avg_rating'].mean()
C

4.500872600349041

So, the mean rating for all the jokes is approx 4.5 on a scale of 10.The next step is to determine an appropriate value for m, the minimum votes required to be listed in the chart. We will use 70th percentile as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 70% of the jokes in the list.

In [11]:
m= df['rating_count'].quantile(0.7)
m

3.0

In [12]:
q_jokes = df.copy().loc[df['rating_count'] >= m]
q_jokes.shape

(191, 11)

We see that there are 191 jokes which qualify to be in this list. Now, we need to calculate our metric for each qualified movie. To do this, we will define a function, weighted_rating() and define a new feature score, of which we'll calculate the value by applying this function to our DataFrame of qualified jokes:

In [13]:
def weighted_rating(x, m=m, C=C):
    v = x['rating_count']
    R = x['avg_rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [14]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_jokes['score'] = q_jokes.apply(weighted_rating, axis=1)

Finally, let's sort the DataFrame based on the score feature and output the title, vote count, vote average and weighted rating or score of the top 10 jokes.

In [15]:
#Sort movies based on score calculated above
q_jokes = q_jokes.sort_values('score', ascending=False)

#Print the top 15 movies
q_jokes[['jokeText', 'rating_count', 'avg_rating', 'score']].head(10)

,jokeText,rating_count,avg_rating,score
105,On the first day God created the cow...\nGod s...,3,9.333333,6.917103
115,Q: How many libertarians does it take to chang...,3,9.333333,6.917103
62,What do you get when you run over a parakeet w...,3,8.666667,6.583770
181,Yo Mama so fat that she has to use a VCR as a...,3,8.666667,6.583770
67,An old Scotsmen is sitting with a younger Scot...,3,7.333333,5.917103
26,What are minorities?\nLesser people.,3,7.333333,5.917103
87,Three engineering students were gathered toget...,3,7.333333,5.917103
186,A concerned husband went to a doctor to talk a...,3,7.333333,5.917103
64,A guy walks into a bar and sits down next to a...,3,7.000000,5.750436
183,"1. Capmbell's Soup Condoms: Mmm, mmmm, good.....",3,7.000000,5.750436
